In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
events = pd.read_csv("events_up_to_01062018.csv", low_memory=False, dtype={'sku' : 'object'})
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)

In [4]:
events.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,2341681,2341681,2341681,191131,1320530,1321513,1320530,1320530,1320530,505949,...,106406,204069,204069,204069,204069,204069,204069,204066,204069,204069
unique,1490912,11,38829,248,3693,208,5,8,63,52267,...,4,7,2,2206,122,51,4,393,131,366
top,2018-05-31 01:59:16,viewed product,c76b8417,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",...,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,14,1248124,4438,64187,8462,107262,547617,442096,314925,2606,...,105195,91753,165827,36866,57304,197699,103502,73234,46648,57953


In [5]:
events.columns.values

array(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'], dtype=object)

## Convertimos los datos categóricos

In [6]:
events['timestamp'] = pd.to_datetime(events['timestamp'], errors = 'coerce', format= '%Y-%m-%d')
events['event'] = pd.Categorical(events['event'])
events['url'] = pd.Categorical(events['url'])
events['sku'] = pd.Categorical(events['sku'])
events['model'] = pd.Categorical(events['model'])
events['condition'] = pd.Categorical(events['condition'])
events['storage'] = pd.Categorical(events['storage'])
events['color'] = pd.Categorical(events['color'])
events['staticpage'] = pd.Categorical(events['staticpage'])
events['campaign_source'] = pd.Categorical(events['campaign_source'])
events['search_engine'] = pd.Categorical(events['search_engine'])
events['channel'] = pd.Categorical(events['channel'])
events['new_vs_returning'] = pd.Categorical(events['new_vs_returning'])
events['city'] = pd.Categorical(events['city'])
events['region'] = pd.Categorical(events['region'])
events['country'] = pd.Categorical(events['country'])
events['device_type'] = pd.Categorical(events['device_type'])
events['screen_resolution'] = pd.Categorical(events['screen_resolution'])
events['operating_system_version'] = pd.Categorical(events['operating_system_version'])
events['browser_version'] = pd.Categorical(events['browser_version'])

## Agregamos columnas

In [7]:
events['day'] = events['timestamp'].dt.day
events['month'] = events['timestamp'].dt.month

In [8]:
events['marca'] = pd.Categorical(events['model'].str.split(' ').str[0])
events['modelo'] = events['model'].str.split(' ').str[1:].str.join(' ')
events[['marca', 'modelo']].head()

,marca,modelo
0,Samsung,Galaxy J7 Prime
1,iPhone,5s
2,iPhone,6S
3,iPhone,7
4,LG,G4 H818P


In [9]:
events['operating_system_name'] = events['operating_system_version'].str.split(' ').str[0]
events['operating_system_name'] = np.where(events['operating_system_version'].str.split(' ').str[1]=='Phone', events['operating_system_name'] + ' Phone', events['operating_system_name'])
events['operating_system_name'] = np.where(events['operating_system_version'].str.split(' ').str[1]=='OS', events['operating_system_name'] + ' OS', events['operating_system_name'])
events['operating_system_name'].value_counts()

Android          96901
Windows          95614
iOS               8312
Windows Phone      917
Mac OS             902
Linux              766
Ubuntu             230
Chrome OS          205
Other              105
BlackBerry OS       74
Tizen               37
Fedora               4
FreeBSD              1
Symbian OS           1
Name: operating_system_name, dtype: int64

In [10]:
events['browser_name'] = events['browser_version'].str.split(' ').str[0]
events['browser_name'] = np.where(events['browser_version'].str.split(' ').str[1]=='Mobile', events['browser_name'] + ' Mobile', events['browser_name'])
events['browser_name'].value_counts()

Chrome            87918
Chrome Mobile     85915
Mobile             7730
Firefox            6777
Samsung            4809
Facebook           3064
Edge               2267
Opera              1548
IE                 1118
Android             946
Edge Mobile         485
IE Mobile           432
UC                  311
Firefox Mobile      227
Safari              226
BlackBerry           73
Chromium             63
Opera Mobile         52
Puffin               26
Vivaldi              25
Yandex               24
Pinterest            13
Other                 9
Maxthon               6
WebKit                3
K-Meleon              1
BingPreview           1
Name: browser_name, dtype: int64

## Usuarios

#### Cantidad de personas

In [11]:
# Cantidad de usuarios únicos
events['person'].nunique()

38829

#### Sesiones por usuario

In [12]:
# Tomo como sesión 1 hora
events = events.sort_values(by='timestamp')
events['timestamp_anterior'] = events.groupby('person')['timestamp'].shift()
events['start_session'] = ((events['timestamp'] - events['timestamp_anterior']) >= timedelta(minutes=60)) | events['timestamp_anterior'].isnull()
events['start_session'] = events['start_session'].astype(int)

In [13]:
events[['start_session', 'timestamp', 'timestamp_anterior', 'person']].head(20)

,start_session,timestamp,timestamp_anterior,person
2307205,1,2018-01-01 08:09:31,NaT,0f4e2a4b
1753202,0,2018-01-01 08:09:31,2018-01-01 08:09:31,0f4e2a4b
1753201,0,2018-01-01 08:09:31,2018-01-01 08:09:31,0f4e2a4b
1753200,0,2018-01-01 08:09:44,2018-01-01 08:09:31,0f4e2a4b
1753207,0,2018-01-01 08:45:29,2018-01-01 08:09:44,0f4e2a4b
2307206,0,2018-01-01 08:45:29,2018-01-01 08:45:29,0f4e2a4b
1753210,0,2018-01-01 08:45:29,2018-01-01 08:45:29,0f4e2a4b
1753203,0,2018-01-01 08:45:43,2018-01-01 08:45:29,0f4e2a4b
1753204,0,2018-01-01 08:48:57,2018-01-01 08:45:43,0f4e2a4b
1753213,0,2018-01-01 08:49:05,2018-01-01 08:48:57,0f4e2a4b


In [14]:
# Ver de usar otro formato para el id de sesión ?
events['session_num'] = events.groupby('person')['start_session'].cumsum()
events['session_id'] = events['session_num'].astype(str) + '-' + events['person']
events['session_id'].head()

2307205    1-0f4e2a4b
1753202    1-0f4e2a4b
1753201    1-0f4e2a4b
1753200    1-0f4e2a4b
1753207    1-0f4e2a4b
Name: session_id, dtype: object

In [15]:
# Para las sesiones ver de buscar tiempo total, inicio-fin, ctd de eventos, etc
# Se podría hacer un nuevo df con eso por id sesión? 

In [15]:
events['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

## Features
### Falta revisar el tp1 y recopilar lo que estaba copado


### De los eventos
Cantidad total de eventos por cada evento: x visited, y viewed, etc

### De las sesiones
Cantidad total de sesiones (con lo de arriba)
Suma de tiempo de todas las sesiones
Promedio de timepo de sesión

### Propio de c/evento
Qué features se pueden sacar de cada evento?
#### viewed product
-marca
-condición
-color
-storage
-sku, como ?

#### brand listing 

#### visited site

#### ad campaign hit 
qué campaña? 
cantidad de veces?
campaign_source
también de url se puede sacar:
##### events['ads_accion'] = events['url'].str.split('/').str[1]
##### events['ads_marca'] = events['url'].str.split('/').str[2]
##### events['ads_modelo'] = events['url'].str.split('/').str[3]

#### generic listing 

#### searched products 
skus que buscaron? cuantos
palabras buscadas, comparar contra marca?

#### search engine hit 
de donde?

#### checkout
me parece que va lo mismo que viewed

#### staticpage

#### conversion
me parece que va lo mismo que viewed

#### lead 
cuantos puso?

## Cantidad de eventos

In [16]:
# pongo los eventos como columnas y cuento la ctd
events_data = events.pivot_table(index='person', columns='event', values='timestamp', aggfunc='count', fill_value=0)
events_data.columns = events_data.columns.astype('object')
events_data.reset_index(inplace=True)
final_data = events_data
events_data.head()

event,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0,0,3,0,1,0,0,0,0,0,2
1,00091926,15,25,2,0,0,0,0,0,0,372,34
2,00091a7a,1,5,0,0,0,0,0,0,0,3,1
3,000ba417,1,24,6,1,14,0,1,0,0,153,6
4,000c79fe,1,0,1,0,1,0,1,9,0,3,1


## Viewed product

In [17]:
viewed_events = events[events['event']=='viewed product'][['timestamp', 'event', 'person', 'sku', 'model', 'condition', 
                                           'storage', 'color', 'day', 'month', 'marca', 'modelo', 'timestamp_anterior', 
                                           'start_session', 'session_num', 'session_id']]

## marcas que vio
viewed_data = viewed_events[['person', 'marca', 'timestamp']].pivot_table(index='person', columns='marca', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()

## condition
viewed_data = viewed_events[['person', 'condition', 'timestamp']].pivot_table(index='person', columns='condition', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()

## color
viewed_data = viewed_events[['person', 'color', 'timestamp']].pivot_table(index='person', columns='color', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()

## storage
viewed_data = viewed_events[['person', 'storage', 'timestamp']].pivot_table(index='person', columns='storage', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()


,person,viewed_128GB,viewed_16GB,viewed_256GB,viewed_32GB,viewed_4GB,viewed_512MB,viewed_64GB,viewed_8GB
0,00091926,48,104,10,132,0,0,78,0
1,00091a7a,1,1,0,0,0,0,1,0
2,000ba417,0,108,1,20,1,0,1,22
3,000c79fe,3,0,0,0,0,0,0,0
4,000e4d9e,1,108,1,208,0,0,21,0


In [18]:
final_data.columns

Index(['person', 'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'viewed_Asus',
       'viewed_LG', 'viewed_Lenovo', 'viewed_Motorola', 'viewed_Outros',
       'viewed_Quantum', 'viewed_Samsung', 'viewed_Sony', 'viewed_Xiaomi',
       'viewed_iPad', 'viewed_iPhone', 'viewed_Bom',
       'viewed_Bom - Sem Touch ID', 'viewed_Excelente', 'viewed_Muito Bom',
       'viewed_Novo', 'viewed_Amarelo', 'viewed_Ametista', 'viewed_Azul',
       'viewed_Azul Escuro', 'viewed_Azul Safira', 'viewed_Azul Topázio',
       'viewed_Bambu', 'viewed_Black Piano', 'viewed_Branco',
       'viewed_Branco Azul', 'viewed_Branco Azul Navy', 'viewed_Branco Bambu',
       'viewed_Branco Cabernet', 'viewed_Branco Dourado',
       'viewed_Branco Framboesa', 'viewed_Branco Pink', 'viewed_Branco Verde',
       'viewed_Branco Vermelho', 'viewed_Cabernet', 'viewed_Cinza',
    

In [19]:
final_data.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,viewed_Vermelho,viewed_Ônix,viewed_128GB,viewed_16GB,viewed_256GB,viewed_32GB,viewed_4GB,viewed_512MB,viewed_64GB,viewed_8GB
0,0008ed71,0,0,3,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00091926,15,25,2,0,0,0,0,0,0,...,2.0,0.0,48.0,104.0,10.0,132.0,0.0,0.0,78.0,0.0
2,00091a7a,1,5,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,000ba417,1,24,6,1,14,0,1,0,0,...,1.0,0.0,0.0,108.0,1.0,20.0,1.0,0.0,1.0,22.0
4,000c79fe,1,0,1,0,1,0,1,9,0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Ad Campaign Hit

In [20]:
campaign_events = events[events['event']=='ad campaign hit'][['timestamp', 'event', 'person', 'url',
       'campaign_source', 'day', 'month', 'timestamp_anterior', 'start_session', 'session_num', 'session_id']]
campaign_events['ads_accion'] = campaign_events['url'].str.split('/').str[1]
campaign_events['ads_marca'] = campaign_events['url'].str.split('/').str[2]
campaign_events['ads_modelo'] = campaign_events['url'].str.split('/').str[3]

## source
campaign_data = campaign_events[campaign_events['campaign_source'].notnull()]
campaign_data = campaign_data[['person', 'campaign_source', 'timestamp']].pivot_table(index='person', columns='campaign_source', values='timestamp', aggfunc='count', fill_value=0)
campaign_data.columns = campaign_data.columns.astype('object')
campaign_data.reset_index(inplace=True)
campaign_data.columns = ['campaign-source_'+col if col != 'person' else col for col in campaign_data.columns]
final_data = pd.merge(final_data, campaign_data, on='person', how='left').fillna(0)
campaign_data.head()

## marca
campaign_data = campaign_events[campaign_events['ads_marca'].notnull()]
campaign_data = campaign_events[['person', 'ads_marca', 'timestamp']].pivot_table(index='person', columns='ads_marca', values='timestamp', aggfunc='count', fill_value=0)
campaign_data.columns = campaign_data.columns.astype('object')
campaign_data.reset_index(inplace=True)
campaign_data.columns = ['campaign-marca_'+col if col != 'person' else col for col in campaign_data.columns]
final_data = pd.merge(final_data, campaign_data, on='person', how='left').fillna(0)
campaign_data.head()

,person,campaign-marca_,campaign-marca_apple,campaign-marca_asus,campaign-marca_ipad,campaign-marca_iphone,campaign-marca_lenovo,campaign-marca_lg,campaign-marca_list,campaign-marca_motorola,campaign-marca_nokia,campaign-marca_ofertas,campaign-marca_quantum,campaign-marca_samsung,campaign-marca_sony
0,00091926,0,0,0,0,8,0,0,0,0,0,0,0,7,0
1,00091a7a,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,000ba417,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,000e4d9e,0,0,0,0,0,0,0,0,0,0,0,0,5,0
4,000e619d,0,0,0,0,1,0,1,0,0,0,0,0,2,0


In [59]:
final_data_t = final_data[['person',
 'ad campaign hit',
 'brand listing',
 'checkout',
 'conversion',
 'generic listing',
 'lead',
 'search engine hit',
 'searched products',
 'staticpage',
 'viewed product',
 'visited site']]

In [60]:
final_data_t.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0,0,3,0,1,0,0,0,0,0,2
1,00091926,15,25,2,0,0,0,0,0,0,372,34
2,00091a7a,1,5,0,0,0,0,0,0,0,3,1
3,000ba417,1,24,6,1,14,0,1,0,0,153,6
4,000c79fe,1,0,1,0,1,0,1,9,0,3,1


In [21]:
final_data.columns.tolist()

['person',
 'ad campaign hit',
 'brand listing',
 'checkout',
 'conversion',
 'generic listing',
 'lead',
 'search engine hit',
 'searched products',
 'staticpage',
 'viewed product',
 'visited site',
 'viewed_Asus',
 'viewed_LG',
 'viewed_Lenovo',
 'viewed_Motorola',
 'viewed_Outros',
 'viewed_Quantum',
 'viewed_Samsung',
 'viewed_Sony',
 'viewed_Xiaomi',
 'viewed_iPad',
 'viewed_iPhone',
 'viewed_Bom',
 'viewed_Bom - Sem Touch ID',
 'viewed_Excelente',
 'viewed_Muito Bom',
 'viewed_Novo',
 'viewed_Amarelo',
 'viewed_Ametista',
 'viewed_Azul',
 'viewed_Azul Escuro',
 'viewed_Azul Safira',
 'viewed_Azul Topázio',
 'viewed_Bambu',
 'viewed_Black Piano',
 'viewed_Branco',
 'viewed_Branco Azul',
 'viewed_Branco Azul Navy',
 'viewed_Branco Bambu',
 'viewed_Branco Cabernet',
 'viewed_Branco Dourado',
 'viewed_Branco Framboesa',
 'viewed_Branco Pink',
 'viewed_Branco Verde',
 'viewed_Branco Vermelho',
 'viewed_Cabernet',
 'viewed_Cinza',
 'viewed_Cinza espacial',
 'viewed_Cobre',
 'viewed_Co

In [22]:
final_data.shape

(38829, 136)

In [23]:
final_data.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,campaign-marca_iphone,campaign-marca_lenovo,campaign-marca_lg,campaign-marca_list,campaign-marca_motorola,campaign-marca_nokia,campaign-marca_ofertas,campaign-marca_quantum,campaign-marca_samsung,campaign-marca_sony
0,0008ed71,0,0,3,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00091926,15,25,2,0,0,0,0,0,0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
2,00091a7a,1,5,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,1,24,6,1,14,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,000c79fe,1,0,1,0,1,0,1,9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
set_entrenamiento = pd.merge(final_data_t, training_labels, on ='person', how = 'inner')
set_entrenamiento.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,label
0,0008ed71,0,0,3,0,1,0,0,0,0,0,2,0
1,000c79fe,1,0,1,0,1,0,1,9,0,3,1,0
2,001802e4,5,0,1,0,4,0,0,4,0,4,1,0
3,0019e639,29,165,15,2,28,0,13,11,0,189,19,0
4,001b0bf9,0,1,2,1,1,0,0,0,0,2,0,0


In [63]:
set_entrenamiento = set_entrenamiento.drop(columns = ['person'])
set_entrenamiento.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,label
0,0,0,3,0,1,0,0,0,0,0,2,0
1,1,0,1,0,1,0,1,9,0,3,1,0
2,5,0,1,0,4,0,0,4,0,4,1,0
3,29,165,15,2,28,0,13,11,0,189,19,0
4,0,1,2,1,1,0,0,0,0,2,0,0


In [64]:
X = set_entrenamiento.drop(columns = ['label'])
y = set_entrenamiento["label"]

In [109]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

In [110]:

data_dmatrix = xgb.DMatrix(data=X,label=y)

In [166]:

xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 7, alpha = 5, n_estimators = 10)

In [167]:
xg_reg.fit(x_train,y_train)

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [168]:
preds = xg_reg.predict(x_test)
preds

array([0.20284909, 0.23509982, 0.1857109 , ..., 0.25274277, 0.19288489,
       0.1857109 ], dtype=float32)

In [169]:
preds = preds.tolist()
df = pd.DataFrame(preds)
df.head()

,0
0,0.202849
1,0.235100
2,0.185711
3,0.191667
4,0.185711


In [170]:
df = df[0].map(lambda x: 1 if x > 0.5 else 0);

In [171]:
accuracy_score(df,y_test)

0.9475920679886686

In [172]:
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)

In [173]:
set_test = pd.merge(final_data_t,labels_predict,on = "person", how = "right")
set_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19415 entries, 0 to 19414
Data columns (total 12 columns):
person               19415 non-null object
ad campaign hit      19415 non-null int64
brand listing        19415 non-null int64
checkout             19415 non-null int64
conversion           19415 non-null int64
generic listing      19415 non-null int64
lead                 19415 non-null int64
search engine hit    19415 non-null int64
searched products    19415 non-null int64
staticpage           19415 non-null int64
viewed product       19415 non-null int64
visited site         19415 non-null int64
dtypes: int64(11), object(1)
memory usage: 1.9+ MB


In [174]:
personas = set_test["person"]
set_test = set_test.drop(columns=["person"])
set_test = set_test.fillna(0)

In [175]:
predicts_kaggel = xg_reg.predict(set_test)
set_test.tail()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
19410,0,0,0,0,0,0,1,0,0,19,1
19411,1,0,1,0,0,0,0,0,0,1,1
19412,0,0,0,0,0,0,1,0,0,0,1
19413,7,1,1,1,12,0,12,8,3,42,10
19414,1,2,1,0,0,0,1,0,0,4,1


In [176]:
predicts_kaggel = predicts_kaggel.tolist()
df_kaggel = pd.DataFrame(predicts_kaggel)
df_kaggel.head()

,0
0,0.223193
1,0.222509
2,0.219284
3,0.231735
4,0.204703


In [177]:
df_kaggel = df_kaggel[0].map(lambda x: 1 if x > 0.5 else 0);
df_kaggel = df_kaggel.to_frame()
df_kaggel["person"] = personas
df_kaggel.rename(columns = {0: 'label'},inplace = True) 

In [178]:
df_kaggel = df_kaggel[['person','label']]

In [179]:
df_kaggel.tail()

,person,label
19410,fff1b11a,0
19411,fff1caee,0
19412,fff54d61,0
19413,fff72025,0
19414,fffd1246,0


In [180]:
df_kaggel.to_csv('Resultados.csv',index=False)

In [181]:
len(df_kaggel)

19415

In [182]:
df_kaggel['label'].value_counts()

0    19414
1        1
Name: label, dtype: int64

In [165]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Columns: 136 entries, person to campaign-marca_sony
dtypes: float64(124), int64(11), object(1)
memory usage: 40.6+ MB
